# Data Pre-Processing

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import pickle

In [2]:
with open('../data/features.csv') as f1:
    df = pd.read_csv(f1)

with open('../data/outcomes.csv') as f2:
    outcomes = pd.read_csv(f2, usecols=['student_id', 'outcome'])
                           
df = pd.merge(df, outcomes, on='student_id')
 

df.drop(['Unnamed: 0', 'course_id'], axis='columns', inplace=True)
labels = ['fail', 'pass']
target_variable_name = 'outcome'
df.dropna(axis='index', how='any', subset=['outcome'], inplace=True)
df.outcome = df.outcome.astype(int)

In [3]:
df.head()

,student_id,tot_page_views,average_daily_views,median_daily_views,max_daily_views,days_with_views,hours_with_views,max_views_per_hour,avg_views_per_hour,median_views_per_hour,avg_hours_with_views_per_day,max_hours_with_views_per_day,median_hours_with_views_per_day,tot_participations,average_daily_participations,median_daily_participations,max_daily_participations,days_with_participations,outcome
0,47649,1153,22.607843,16.0,118.0,51,71,88.0,16.239437,15.0,1.392157,1.392157,1.0,23,0.460000,0.0,4.0,50,1
1,52263,744,15.183673,0.0,93.0,49,67,48.0,11.104478,8.0,1.367347,1.367347,0.0,20,0.434783,0.0,3.0,46,1
2,24036,341,6.686275,0.0,80.0,51,27,48.0,12.629630,11.0,0.529412,0.529412,0.0,15,0.333333,0.0,3.0,45,1
3,52137,237,5.042553,0.0,68.0,47,19,32.0,12.473684,9.0,0.404255,0.404255,0.0,6,0.187500,0.0,3.0,32,1
4,52267,665,13.300000,0.0,161.0,50,42,118.0,15.833333,8.0,0.840000,0.840000,0.0,26,0.520000,0.0,5.0,50,1


## Missing Values

Since there are only a few missing values we can simply discard them

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 422 entries, 0 to 424
Data columns (total 19 columns):
 #   Column                           Non-Null Count  Dtype  
---  ------                           --------------  -----  
 0   student_id                       422 non-null    int64  
 1   tot_page_views                   422 non-null    int64  
 2   average_daily_views              419 non-null    float64
 3   median_daily_views               419 non-null    float64
 4   max_daily_views                  419 non-null    float64
 5   days_with_views                  422 non-null    int64  
 6   hours_with_views                 422 non-null    int64  
 7   max_views_per_hour               419 non-null    float64
 8   avg_views_per_hour               419 non-null    float64
 9   median_views_per_hour            419 non-null    float64
 10  avg_hours_with_views_per_day     419 non-null    float64
 11  max_hours_with_views_per_day     419 non-null    float64
 12  median_hours_with_view

In [5]:
df.dropna(axis='index', how='any', inplace=True)

In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 416 entries, 0 to 424
Data columns (total 19 columns):
 #   Column                           Non-Null Count  Dtype  
---  ------                           --------------  -----  
 0   student_id                       416 non-null    int64  
 1   tot_page_views                   416 non-null    int64  
 2   average_daily_views              416 non-null    float64
 3   median_daily_views               416 non-null    float64
 4   max_daily_views                  416 non-null    float64
 5   days_with_views                  416 non-null    int64  
 6   hours_with_views                 416 non-null    int64  
 7   max_views_per_hour               416 non-null    float64
 8   avg_views_per_hour               416 non-null    float64
 9   median_views_per_hour            416 non-null    float64
 10  avg_hours_with_views_per_day     416 non-null    float64
 11  max_hours_with_views_per_day     416 non-null    float64
 12  median_hours_with_view

## Feature Selection


In [7]:
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split, cross_val_score, StratifiedKFold
from sklearn.linear_model import LogisticRegression
from sklearn.feature_selection import SelectFromModel, RFECV
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

In [8]:
x = df.drop(['student_id','outcome'], axis='columns')
y = df['outcome'].values.reshape(-1,1)

In [9]:
x.shape, y.shape

((416, 17), (416, 1))

In [10]:
Min_Max = MinMaxScaler()
X = Min_Max.fit_transform(x)
Y= Min_Max.fit_transform(y)

# Split the data into test and training
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.3, random_state=0)

X_train.shape, X_test.shape

((291, 17), (125, 17))

In [11]:
scores_df = pd.DataFrame()
# Create a naive random forest classifier
clf = RandomForestClassifier(n_estimators=1000, random_state=0, n_jobs=-1)
# Train the classifier
cv_scores = cross_val_score(clf, X_train,np.ravel(Y_train), cv=10)

print("Naive Random Forest all features")
print(f"Mean Accuracy: {np.mean(cv_scores)} \nStandard Deviation : {np.std(cv_scores)}")

Naive Random Forest all features
Mean Accuracy: 0.8213793103448277 
Standard Deviation : 0.052653715852762434


In [12]:
scores_df = scores_df.append({'# features':'all',
               'Mean Accuracy':np.mean(cv_scores),
               'Std':np.std(cv_scores)}, ignore_index=True)

In [13]:
scores_df

,# features,Mean Accuracy,Std
0,all,0.821379,0.052654


### Lasso

In [14]:
sel = SelectFromModel(LogisticRegression(C=1, penalty='l1', solver='liblinear'))
sel.fit(X_train, np.ravel(Y_train))

SelectFromModel(estimator=LogisticRegression(C=1, penalty='l1',
                                             solver='liblinear'))

In [15]:
print("Selected features :")
_ = [print(f"- {c}") for c in x.loc[:, sel.get_support()].columns]

Selected features :
- hours_with_views
- avg_views_per_hour
- tot_participations
- days_with_participations


In [16]:
X_train_df = pd.DataFrame(X_train)
selected_features = X_train_df.columns[(sel.get_support())]
print(f'total features: {(X_train_df.shape[1])}')
print(f'selected features: {len(selected_features)}')
print(f'features with coefficients shrank to zero: {np.sum(sel.estimator_.coef_ == 0)}')

total features: 17
selected features: 4
features with coefficients shrank to zero: 13


In [17]:
removed_features = X_train_df.columns[(sel.estimator_.coef_ == 0).ravel().tolist()]
removed_features

Int64Index([0, 1, 2, 3, 4, 6, 8, 9, 10, 11, 13, 14, 15], dtype='int64')

In [18]:
X_train_lasso_selected = sel.transform(X_train_df)
X_test_lasso_selected = sel.transform(X_test)

In [19]:
# Create a naive random forest classifier
#clf = RandomForestClassifier(n_estimators=1000, random_state=0, n_jobs=-1)
# Train the classifier
cv_scores = cross_val_score(clf, X_train_lasso_selected, np.ravel(Y_train), cv=10)

print("Naive Random Forest all features")
print(f"Mean Accuracy: {np.mean(cv_scores)} \nStandard Deviation : {np.std(cv_scores)}")

Naive Random Forest all features
Mean Accuracy: 0.8317241379310346 
Standard Deviation : 0.06587560792419064


In [20]:
scores_df = scores_df.append({'# features':'lasso',
               'Mean Accuracy':np.mean(cv_scores),
               'Std':np.std(cv_scores)}, ignore_index=True)
scores_df.head()

,# features,Mean Accuracy,Std
0,all,0.821379,0.052654
1,lasso,0.831724,0.065876


### RFE

In [21]:
# Load/Run RFE
pkl_filename = "../models/rfecv.pkl"
try :
    # Load Model
    rfecv = pickle.load(open(pkl_filename, 'rb'))

except :
    # Perform RFE
    print('Training')
    rfecv = RFECV(estimator=clf, step=1, cv=StratifiedKFold(10), scoring='accuracy')
    rfecv.fit(X_train, np.ravel(Y_train))
    # Save model
    with open('../models/rfecv.pkl', 'wb') as file:
        pickle.dump(rfecv, file)

In [22]:
print(f'Optimal number of features: {rfecv.n_features_}')

Optimal number of features: 8


In [23]:
print("Selected features :")
_ = [print(f"- {c}") for c in x.loc[:, rfecv.support_].columns]

Selected features :
- tot_page_views
- days_with_views
- hours_with_views
- avg_views_per_hour
- avg_hours_with_views_per_day
- tot_participations
- average_daily_participations
- days_with_participations


In [24]:
plt.figure(figsize=(12, 6))
plt.title('Recursive Feature Elimination with Cross-Validation', fontsize=20)
plt.xlabel('Number of features selected', fontsize=14, labelpad=20)
plt.ylabel('% Correct Classification', fontsize=14, labelpad=20)
plt.plot(range(1, len(rfecv.grid_scores_) + 1), rfecv.grid_scores_, 'bo-', linewidth=3)

plt.show()

In [25]:
X_train_rfe_selected = rfecv.transform(X_train_df)
X_test_rfe_selected = rfecv.transform(X_test)

In [26]:
dset = pd.DataFrame()
dset['attr'] = x.loc[:, rfecv.support_].columns
dset['importance'] = rfecv.estimator_.feature_importances_

dset = dset.sort_values(by='importance', ascending=True)


plt.figure(figsize=(12, 7))
plt.barh(y=dset['attr'], width=dset['importance'], color='#1976D2')
plt.title('RFECV - Feature Importances', fontsize=20, pad=20)
plt.xlabel('Importance', fontsize=14, labelpad=20)
plt.show()

In [ ]:
# Create a naive random forest classifier
#clf = RandomForestClassifier(n_estimators=1000, random_state=0, n_jobs=-1)
# Train the classifier
cv_scores = cross_val_score(clf, X_train_rfe_selected, np.ravel(Y_train), cv=10)

print("Naive Random Forest all features")
print(f"Mean Accuracy: {np.mean(cv_scores)} \nStandard Deviation : {np.std(cv_scores)}")

In [ ]:
scores_df = scores_df.append({'# features':'rfe',
               'Mean Accuracy':np.mean(cv_scores),
               'Std':np.std(cv_scores)}, ignore_index=True)
scores_df.head()

In [ ]:
plt.errorbar(scores_df['# features'], scores_df['Mean Accuracy'], yerr=scores_df['Std'], fmt="o:")
plt.title('Feature Selection Strategy Comparison')
plt.show()

## Modeling

In [ ]:
# Let's use the result of the Recursive Feature Elimination 
X_train = X_train_rfe_selected.copy()
#np.ravel(Y_train)
X_test = X_test_rfe_selected.copy()
y_test = Y_test.copy()

## Bayesian Search CV: XGB and RF compared

In [ ]:
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score
import sklearn.metrics as metrics
#import joblib


from timeit import default_timer as timer
from datetime import datetime, date
from tqdm.auto import tqdm

from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import confusion_matrix
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score
from sklearn.metrics import make_scorer

from skopt import BayesSearchCV
from skopt.space import Real, Categorical, Integer


In [ ]:
def plot_multiclass_confusion_matrix(y_test, y_pred, categories):
    """Plots the confusion matrix given the data and the lables."""
    
    cm = confusion_matrix(y_test, y_pred)
    #print(cm)
    
    group_counts = ["{0:0.0f}\n".format(value) for value in cm.flatten()]
    group_percentages = ["{0:.2%}".format(value) for value in cm.flatten()/np.sum(cm)]
    stats_text = "\n\nAccuracy={:0.2f}%".format(100*np.trace(cm) / float(np.sum(cm)))
    
    box_labels = [f"{v2}{v3}".strip() for v2, v3 in zip(group_counts,group_percentages)]
    box_labels = np.asarray(box_labels).reshape(cm.shape[0],cm.shape[1])

    fig = plt.figure(figsize=(6,6))
    sns.set(font_scale=1.4)
    ax = sns.heatmap(cm, annot=box_labels, fmt='', cmap='Reds', xticklabels=categories, yticklabels=categories)
    ax.set_ylabel('True Label')
    ax.set_xlabel('Predicted Label')
    ax.set_title(stats_text)
    
    return fig, ax

In [ ]:
# Define Search parameters
scoring='precision'
n_iter=10
n_points=1
cv=3


## Xtreme Gradient Boosting

In [ ]:
# train XGB
pkl_filename = f"../models/XGBClassifier_{scoring}.pkl"
try :
    # Load Model
    xgb_opt = pickle.load(open(pkl_filename, 'rb'))

except :
    # Train Model
    param_grid = {'learning_rate':Real(1e-2,1e+0,prior='log-uniform'),
                'n_estimators':Integer(100,1000,prior='uniform'),
                'max_depth':Integer(3,10,prior='uniform'),
                'subsample':Real(0.8,1.0,prior='uniform'),
                'gamma':Integer(0,5,prior='uniform')}

    model = XGBClassifier()
    print("Searching best parameters for "+ str(model))
    print("Start Time : "+ str(datetime.now()))
    xgb_opt = BayesSearchCV(estimator=model,
                         search_spaces=param_grid,
                         scoring=scoring,
                         n_jobs=-1,
                         n_iter=n_iter,
                         n_points=n_points,
                         random_state=0,
                         cv=cv,
                         verbose=0,
                         refit=True,
                         return_train_score = True)
    # perform bayesian optimization
    print("Start Time : "+ str(datetime.now()))
    xgb_opt.fit(X_train, Y_train)
    print("End Time : "+ str(datetime.now()))
    # Save model
    with open(pkl_filename, 'wb') as file:
        pickle.dump(xgb_opt, file)
    
print(pkl_filename)
print("|----Best Precision Score:" + str(xgb_opt.best_score_))
print("|----Best Parameters: " + str(xgb_opt.best_params_))


# XGB
# confusion matrix
Y_pred = xgb_opt.predict(X_test)
_ = plot_multiclass_confusion_matrix(Y_test, Y_pred, ['fail','pass'])

In [ ]:
# predict probabilities
probs_xgb = xgb_opt.predict_proba(X_test)
# calculate the fpr and tpr for all thresholds of the classification
preds_xgb = probs_xgb[:, 1]
fpr_xgb, tpr_xgb, threshold_xgb = metrics.roc_curve(Y_test, preds_xgb)
roc_auc_xgb = metrics.auc(fpr_xgb,tpr_xgb)
print(f'AUC_ROC = {round(roc_auc_xgb,4)}')
# find optimal threshold
t = threshold_xgb[np.argmax(tpr_xgb - fpr_xgb)]
print(f'Optimal threshold = {t}')
Y_pred = (probs_xgb[:, 1] >= t).astype(int)
plot_multiclass_confusion_matrix(Y_test, Y_pred, ['fail','pass'])

## Random Forest

In [ ]:
# train RF
pkl_filename = f"../models/RandomForestClassifier_{scoring}.pkl"
try :
    # Load Model
    rf_opt = pickle.load(open(pkl_filename, 'rb'))

except :
    # Train Model
    param_grid = {'n_estimators':Integer(100,1000,prior='uniform'),
                'max_depth':Integer(3,10,prior='uniform'),
                'min_samples_split':Integer(2,10,prior='uniform'),
                'min_samples_leaf':Integer(1,10,prior='uniform'),  
                'class_weight':Categorical(['balanced'])}

    model = RandomForestClassifier()
    print("Searching best parameters for "+ str(model))
    print("Start Time : "+ str(datetime.now()))
    rf_opt = BayesSearchCV(estimator=model,
                         search_spaces=param_grid,
                         scoring=scoring,
                         n_jobs=-1,
                         n_iter=n_iter,
                         n_points=n_points,
                         random_state=0,
                         cv=cv,
                         verbose=0,
                         refit=True,
                         return_train_score = True)
    # perform bayesian optimization
    print("Start Time : "+ str(datetime.now()))
    rf_opt.fit(X_train, Y_train)
    print("End Time : "+ str(datetime.now()))
    # Save model
    with open(pkl_filename, 'wb') as file:
        pickle.dump(rf_opt, file)
    
print(pkl_filename)
print("|----Best Precision Score:" + str(rf_opt.best_score_))
print("|----Best Parameters: " + str(rf_opt.best_params_))

# RF
# confusion matrix
Y_pred = rf_opt.predict(X_test)
_ = plot_multiclass_confusion_matrix(Y_test, Y_pred, ['fail','pass'])

In [ ]:
# predict probabilities
probs_rf = rf_opt.predict_proba(X_test)
# calculate the fpr and tpr for all thresholds of the classification
preds_rf = probs_rf[:, 1]
fpr_rf, tpr_rf, threshold_rf = metrics.roc_curve(Y_test, preds_rf)
roc_auc_rf = metrics.auc(fpr_rf,tpr_rf)
print(f'AUC_ROC = {round(roc_auc_rf,4)}')
# find optimal threshold
t = threshold_rf[np.argmax(tpr_rf - fpr_rf)]
print(f'Optimal threshold = {t}')
Y_pred = (probs_xgb[:, 1] >= t).astype(int)
plot_multiclass_confusion_matrix(Y_test, Y_pred, ['fail','pass'])

In [ ]:
# look at confusion matrix
# minimize erros on failure
# 70% errors basically data imblance... so maybe balance data?

## Comparing ROC - AUC

In [ ]:
plt.title('Receiver Operating Characteristic')
plt.plot(fpr_rf, tpr_rf, 'g', label = f'RF - AUC = {round(roc_auc_rf,3)}')
plt.plot(fpr_xgb, tpr_xgb, 'b', label = f'XGB - AUC = {round(roc_auc_xgb,3)}')
plt.plot([0, 1], [0, 1],'k:', label='No Skill')
plt.legend(loc = 'lower right')
plt.xlim([0, 1])
plt.ylim([0, 1])
plt.ylabel('True Positive Rate')
plt.xlabel('False Positive Rate')
plt.show()
